In [2]:
# Spark Session
from pyspark.sql import SparkSession


spark = (
    SparkSession
    .builder
    .appName("Spark Introduction")
    .master("local[*]")
    .getOrCreate()
)
spark

In [47]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"],
    ["021","102","Grace Kim","32","","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"


# Create emp DataFrame

emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [49]:
emp_df.printSchema()
emp_df.show(21)

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|       

In [51]:
from pyspark.sql.functions import when,col,lower,expr

emp_gender_fixed = emp_df.withColumn("G",when(lower(col("gender"))=="male",'M')\
                                    .when(lower(col("gender"))=="female",'F')\
                                     .otherwise(None))
emp_gender_fixed.show(21)

# Using expr 
emp_df.withColumn("new_gender",expr("case when gender == 'Male' then 'M' when gender=='Female' then 'F' else null end")).show(5)

+-----------+-------------+-------------+---+------+------+----------+----+
|employee_id|department_id|         name|age|gender|salary| hire_date|   G|
+-----------+-------------+-------------+---+------+------+----------+----+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|   M|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|   F|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|   M|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|   F|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|   M|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|   F|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|   M|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|   F|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|   M|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|   F|
|        011

In [37]:
# Convert data
from pyspark.sql.functions import to_date

emp_date_fix = emp_df.withColumn("hire_date",to_date("hire_date","yyyy-MM-dd"))
emp_date_fix.printSchema()
emp_date_fix.show(5)

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: date (nullable = true)

+-----------+-------------+----------+---+------+------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|
+-----------+-------------+----------+---+------+------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|
+-----------+-------------+----------+---+------+------+----------+
only showing top 5 rows



In [52]:
# Add Date Columns
# Add current data, timestamp,extract year from hire date
from pyspark.sql.functions import current_date,current_timestamp

emp_dated = emp_df.withColumn("date_now",current_date())\
            .withColumn("timestamp_now",current_timestamp())
emp_dated.show(truncate=False)



+-----------+-------------+-------------+---+------+------+----------+----------+--------------------------+
|employee_id|department_id|name         |age|gender|salary|hire_date |date_now  |timestamp_now             |
+-----------+-------------+-------------+---+------+------+----------+----------+--------------------------+
|001        |101          |John Doe     |30 |Male  |50000 |2015-01-01|2025-01-05|2025-01-05 07:17:21.960975|
|002        |101          |Jane Smith   |25 |Female|45000 |2016-02-15|2025-01-05|2025-01-05 07:17:21.960975|
|003        |102          |Bob Brown    |35 |Male  |55000 |2014-05-01|2025-01-05|2025-01-05 07:17:21.960975|
|004        |102          |Alice Lee    |28 |Female|48000 |2017-09-30|2025-01-05|2025-01-05 07:17:21.960975|
|005        |103          |Jack Chan    |40 |Male  |60000 |2013-04-01|2025-01-05|2025-01-05 07:17:21.960975|
|006        |103          |Jill Wong    |32 |Female|52000 |2018-07-01|2025-01-05|2025-01-05 07:17:21.960975|
|007        |101   

In [ ]:
# drop nulls

emp_1 = emp_gender_fixed.drop